# Meta Analysis for GWAS summary statistics

By Malia Schomaker

## Importing and Cleaning the Data

In [36]:
########libraries

# install package if you have not
for (package in c("tidyverse", "metafor", "data.table", "tidyr", "dplyr")) {
  if (package %in% rownames(installed.packages())) {
    library(package, character.only = TRUE)
  } else {
    cat(package, "package is not installed.\n")
    install.packages(package)
    library(package, character.only = TRUE)
  }
}
print("packages done")

####### import data
SNP_data <- read.csv("/gpfs/gibbs/project/bdsi/bdsi_kc2587/lo/new/4_LD_clumping/union_lead_snps.csv", header = TRUE) # sigfig SNPs form LD clumping
og_CAD <- read.csv("/gpfs/gibbs/project/bdsi/bdsi_kc2587/lo/new/1_CAD/CAD_clean.csv", header=TRUE) # full, cleaned CAD GWAS summary
og_MDD <- read.csv("/gpfs/gibbs/project/bdsi/bdsi_kc2587/lo/new/2_MDD/MDD_clean.csv", header=TRUE)# full, cleaned MDD GWAS summary

print("data read")

[1] "packages done"
[1] "data read"


### Pulling the SNPs identified by LD Clumping from the Full dataset

In [37]:
## Merging the significant SNPs (as determiend by LD CLumping) with their CHR, BETA, BP, and SE from the full, clean dataset
intersect_data_CAD <- left_join(SNP_data, og_CAD %>% select(CHR, SNP, BP, BETA.CAD=BETA, SE.CAD=SE_BETA), by = "SNP")
intersect_data_MDD <- left_join(SNP_data, og_MDD %>% select(CHR, SNP, BP, BETA.MDD=BETA, SE.MDD=SE_BETA), by = "SNP")

# checking
head(intersect_data_CAD)
nrow(intersect_data_CAD)
head(intersect_data_MDD)
nrow(intersect_data_MDD)

# there should be 396 SNPs in each

,SNP,CHR,BP,BETA.CAD,SE.CAD
,<chr>,<int>,<int>,<dbl>,<dbl>
1,rs2412546,15,41006523,0.000549,0.0093854
2,rs17613838,11,113638177,0.016465,0.0130940
3,rs2552527,2,218688596,0.051188,0.0096983
4,rs9851130,3,54435589,0.006110,0.0105858
5,rs1975488,4,90984432,0.007721,0.0097244
6,rs10744777,12,112233018,-0.045736,0.0102545


[1] 396

,SNP,CHR,BP,BETA.MDD,SE.MDD
,<chr>,<int>,<int>,<dbl>,<dbl>
1,rs2412546,15,41006523,-0.0226026252,0.0044
2,rs17613838,11,113638177,-0.0285949669,0.0056
3,rs2552527,2,218688596,-0.0009004052,0.0043
4,rs9851130,3,54435589,0.0281014301,0.0049
5,rs1975488,4,90984432,0.0209989426,0.0044
6,rs10744777,12,112233018,0.0060985585,0.0048


[1] 396

### Merging and longating the Dataframes

In [38]:
############# Merge to get BETA2 and SE_beta2 from MDD to CAD
merged_data <- merge(union_data_CAD, union_data_MDD, by = c("CHR", "SNP", "BP"), all = FALSE)

# set as a data table
setDT(merged_data)

head(merged_data)
nrow(merged_data)

CHR,SNP,BP,BETA.CAD,SE.CAD,BETA.MDD,SE.MDD
<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,rs10779284,217092655,0.011537,0.0092844,0.018497855,0.0042
1,rs10874124,80833454,-0.007670,0.0098558,-0.021601645,0.0046
1,rs10922298,197811566,-0.004307,0.0092497,0.025994960,0.0043
1,rs11206510,55496039,0.074519,0.0133438,-0.001901807,0.0054
1,rs11485595,38411350,-0.048311,0.0100502,0.003404199,0.0043
1,rs11591147,55505647,0.256502,0.0572588,-0.025399863,0.0156


[1] 396

In [39]:
############## combined dataframe
# create a long frame with only the SNP, BETA, and SE columns. The SNP column has two copies of each SNP that associate with either their
# CAD BETA and SE or their MDD BETA and SE.
df_long <- rbind(
  merged_data[, .(SNP, BETA=BETA.CAD, SE=SE.CAD)],
  merged_data[!is.na(BETA.MDD) & !is.na(SE.MDD), .(SNP, BETA = BETA.MDD, SE = SE.MDD)]
)

print("combined data frame done")
head(df_long)

[1] "combined data frame done"


SNP,BETA,SE
<chr>,<dbl>,<dbl>
rs10779284,0.011537,0.0092844
rs10874124,-0.007670,0.0098558
rs10922298,-0.004307,0.0092497
rs11206510,0.074519,0.0133438
rs11485595,-0.048311,0.0100502
rs11591147,0.256502,0.0572588


In [14]:
df_long[df_long$SNP == "rs10779284",] # double checking to make sure the SNP repeats and maps to both CAD and MDD

SNP,BETA,SE
<chr>,<dbl>,<dbl>
rs10779284,0.01153700,0.0092844
rs10779284,0.01849785,0.0042000


## Running the Fixed Effect Models

In [13]:
############## running fixed effect modules
meta_per_snp <- function(snp_data) {
  tryCatch({
    res <- rma(yi = BETA, sei = SE, data = snp_data, method = "FE")
    data.frame(
      snp = unique(snp_data$SNP),
      beta_random = as.numeric(res$b),
      se_random = res$se,
      pval_random = res$pval,
      tau_random = res$tau2
    )
  }, error = function(e) {
    message("Meta-analysis failed for SNP: ", unique(snp_data$SNP))
    data.frame(
      snp = unique(snp_data$SNP),
      beta_fixed = NA,
      se_fixed = NA,
      pval_fixed = NA,
      tau_random = NA
    )
  })
}


# Apply to each SNP
res_meta_fixed <- df_long %>%
  group_by(SNP) %>%
  group_split() %>%
  lapply(meta_per_snp) %>%
  bind_rows()

print("fixed effects ran")

############## writing fixed modules to file
# renaming columns for uniformity
colnames(res_meta_fixed) <- c("SNP", "BETA", "SE", "Pval", "Tau")

#arranging by pVal
res_meta_fixed <- res_meta_fixed %>%
    arrange(Pval)

#creating large dataframe with BP and CHR (missing information after running fixed effect models
res_meta_fixed_data <- left_join(res_meta_fixed, merged_data %>% select(CHR, BP, SNP), by = "SNP")

#write this frame to a txt file
fwrite(res_meta_fixed_data, "res_meta_fixed.txt", sep = "\t")

[1] "fixed effects ran"


ERROR: [1m[33mError[39m in `left_join()`:[22m
[1m[22m[33m![39m Join columns in `y` must be present in the data.
[31m✖[39m Problem with `SNP`.


In [ ]:
res_meta_fixed_data[res_meta_fixed_data$SNP == "rs2071382",]

## Running Random Effects Model

In [25]:
############### random-effect model 
meta_per_snp <- function(snp_data) {
  tryCatch({
    res <- rma(yi = BETA, sei = SE, data = snp_data, method = "DL")
    data.frame(
      snp = unique(snp_data$SNP),
      beta_random = as.numeric(res$b),
      se_random = res$se,
      pval_random = res$pval,
      tau_random = res$tau2
    )
  }, error = function(e) {
    message("Meta-analysis failed for SNP: ", unique(snp_data$SNP))
    data.frame(
      snp = unique(snp_data$SNP),
      beta_fixed = NA,
      se_fixed = NA,
      pval_fixed = NA,
      tau_random = NA
    )
  })
}

# Apply to each SNP
res_meta_rand <- df_long %>%
  group_by(SNP) %>%
  group_split() %>%
  lapply(meta_per_snp) %>%
  bind_rows()
print("random effects ran")

############## writing fixed modules to file
# renaming columns for uniformity
colnames(res_meta_rand) <- c("SNP", "BETA", "SE", "Pval", "Tau")

#arranging by pVal
res_meta_rand <- res_meta_rand %>%
    arrange(Pval)

#creating large dataframe with BP and CHR (missing information after running fixed effect models
res_meta_rand_data <- left_join(res_meta_rand, merged_data %>% select(CHR, BP, SNP), by = "SNP")

#write this frame to a txt file
fwrite(res_meta_rand_data, "res_meta_rand.txt", sep = "\t")

[1] "random effects ran"


In [ ]:
## Step 4. Which model to choose?
#For meta-analysis across large-scale GWASs, the huge number of SNPs makes it computationally impossible to
#check the heterogeneity across studies for each SNP. Therefore, the fixed-effect model is generally used. 
#Another justification is that GWAS studies are designed in the same way so the heterogeneity might be minimal. 